In [1]:
import pandas as pd
import numpy as np
import math
import json
%matplotlib inline

In [2]:
# read in the json files
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)

In [3]:
portfolio.shape, profile.shape, transcript.shape

((10, 6), (17000, 5), (306534, 4))

In [4]:
portfolio

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


In [5]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [6]:
transcript.sort_index().head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


# Format and Merge offer portfolio, customer profile, and transcript data

In [7]:
portfolio.rename(columns = {'id': 'offer_id'}, inplace = True)
profile.rename(columns = {'id': 'person'}, inplace = True)

In [8]:
# create dummy columns for the channels
# initiate dummy variables. If created in the following for loop, it is created as a float rather than int. 
for c in ['web', 'email', 'mobile', 'social']:
    portfolio[c] = 0
    
for i in range(len(portfolio)):
    for c in ['web', 'email', 'mobile', 'social']:
        if c in portfolio.loc[i, 'channels']:
            portfolio.loc[i, c] = 1

In [9]:
portfolio.drop('channels', axis = 1, inplace = True)

In [10]:
portfolio

,reward,difficulty,duration,offer_type,offer_id,web,email,mobile,social
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0,1,1,1
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,1,0
3,5,5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,1,0
4,5,20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,1,0,0
5,3,7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,1,1,1,1
6,2,10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,1,1,1,1
7,0,0,3,informational,5a8bc65990b245e5a138643cd4eb9837,0,1,1,1
8,5,5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,1,1,1,1
9,2,10,7,discount,2906b810c7d4411798c6938adc9daaa5,1,1,1,0


In [11]:
# split transcript['value'] column
# can also use "pd.json_normalize(transcript['value'])" to split the dictionary column, just slightly slower
value = pd.DataFrame(transcript['value'].values.tolist(), index=transcript.index)

In [12]:
value.sample(n = 5)

,offer id,amount,offer_id,reward
134778,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
86237,2298d6c36e964ae4a3e7e9706d1fb8c2,NaN,NaN,NaN
123595,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
162483,3f207df678b143eea3cee63160fa8bed,NaN,NaN,NaN
222412,NaN,6.83,NaN,NaN


In [13]:
# concatenate 'offer id'  with 'offer_id' to create a whole list of offers
value['offer'] = value['offer id'].str.cat(value['offer_id'], join='outer', na_rep = '')

In [14]:
# for those rows with no offers, set the offer column to be missing
value.loc[(value['offer id'].isnull()) & (value['offer_id'].isnull()), 'offer'] = np.nan

In [15]:
value.drop(['offer id', 'offer_id'], axis = 1, inplace = True)
value.rename(columns = {'offer': 'offer_id'}, inplace = True)

In [16]:
# merge transcript and value by index 
transcript2 = transcript.join(value)
transcript2.drop('value', axis = 1, inplace = True)

transcript2.shape

(306534, 6)

In [17]:
# reorder the columns
transcript2 = transcript2[['person', 'event', 'offer_id', 'time', 'amount', 'reward']]

In [18]:
# merge transcript with customer profile
trans_profile = transcript2.merge(profile, on = 'person', how = 'outer')

In [19]:
trans_profile.head()

,person,event,offer_id,time,amount,reward,gender,age,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,F,75,20170509,100000.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,F,75,20170509,100000.0
2,78afa995795e4d85b5d9ceeca43f5fef,transaction,NaN,132,19.89,NaN,F,75,20170509,100000.0
3,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,132,NaN,5.0,F,75,20170509,100000.0
4,78afa995795e4d85b5d9ceeca43f5fef,transaction,NaN,144,17.78,NaN,F,75,20170509,100000.0


In [20]:
# merge trans_profile with offer portfolio
data = trans_profile.merge(portfolio, on = 'offer_id', how = 'outer')

In [21]:
data.shape

(306534, 18)

In [22]:
data.head()

,person,event,offer_id,time,amount,reward_x,gender,age,became_member_on,income,reward_y,difficulty,duration,offer_type,web,email,mobile,social
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,0,NaN,NaN,F,75,20170509,100000.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0
1,78afa995795e4d85b5d9ceeca43f5fef,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,6,NaN,NaN,F,75,20170509,100000.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0
2,78afa995795e4d85b5d9ceeca43f5fef,offer completed,9b98b8c7a33c4b65b9aebfe6a799e6d9,132,NaN,5.0,F,75,20170509,100000.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0
3,e2127556f4f64592b11af22de27a7932,offer received,9b98b8c7a33c4b65b9aebfe6a799e6d9,408,NaN,NaN,M,68,20180426,70000.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0
4,e2127556f4f64592b11af22de27a7932,offer viewed,9b98b8c7a33c4b65b9aebfe6a799e6d9,420,NaN,NaN,M,68,20180426,70000.0,5.0,5.0,7.0,bogo,1.0,1.0,1.0,0.0


In [23]:
data[['reward_x', 'reward_y']].count()

reward_x     33579
reward_y    167581
dtype: int64

In [24]:
data.loc[data['reward_x'] != data['reward_y'], 'reward_x'].value_counts()

Series([], Name: reward_x, dtype: int64)

In [25]:
data.drop('reward_x', axis = 1, inplace = True)
data.rename(columns = {'reward_y' : 'reward'}, inplace = True)

In [26]:
# reorder columns
data = data[['event', 'time', 'amount', 'person', 'gender', 'age', 'became_member_on', 'income',
 'offer_id', 'offer_type', 'web', 'email', 'mobile', 'social', 'reward', 'difficulty', 'duration']]

In [27]:
data.sample(n = 5)

,event,time,amount,person,gender,age,became_member_on,income,offer_id,offer_type,web,email,mobile,social,reward,difficulty,duration
257182,offer completed,366,NaN,4fda52477b9c4371b1383c21047e46ab,F,68,20160717,109000.0,fafdcd668e3743c1bb461111dcafc2a4,discount,1.0,1.0,1.0,1.0,2.0,10.0,10.0
179460,offer viewed,0,NaN,3add99f440924505bead6fcc818445ba,F,72,20160721,115000.0,ae264e3637204a6fb9bb56bc8210ddfd,bogo,0.0,1.0,1.0,1.0,10.0,10.0,7.0
13770,offer completed,534,NaN,af58b21715ae4b128897d70ebe49b3f0,M,47,20160324,37000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0
35312,transaction,168,3.36,b3f7699653c64271b2c5d09c928a0af5,M,93,20180129,41000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138748,transaction,570,0.43,df1f60e8aa4c42ad99854c179ab4ff71,None,118,20171217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 306534 entries, 0 to 306533
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   event             306534 non-null  object 
 1   time              306534 non-null  int64  
 2   amount            138953 non-null  float64
 3   person            306534 non-null  object 
 4   gender            272762 non-null  object 
 5   age               306534 non-null  int64  
 6   became_member_on  306534 non-null  int64  
 7   income            272762 non-null  float64
 8   offer_id          167581 non-null  object 
 9   offer_type        167581 non-null  object 
 10  web               167581 non-null  float64
 11  email             167581 non-null  float64
 12  mobile            167581 non-null  float64
 13  social            167581 non-null  float64
 14  reward            167581 non-null  float64
 15  difficulty        167581 non-null  float64
 16  duration          16

In [29]:
# convert became_member_on to datetime object
data['became_member_on'] =  pd.to_datetime(data['became_member_on'], format = '%Y%m%d')

In [30]:
data.head()

,event,time,amount,person,gender,age,became_member_on,income,offer_id,offer_type,web,email,mobile,social,reward,difficulty,duration
0,offer received,0,NaN,78afa995795e4d85b5d9ceeca43f5fef,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0
1,offer viewed,6,NaN,78afa995795e4d85b5d9ceeca43f5fef,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0
2,offer completed,132,NaN,78afa995795e4d85b5d9ceeca43f5fef,F,75,2017-05-09,100000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0
3,offer received,408,NaN,e2127556f4f64592b11af22de27a7932,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0
4,offer viewed,420,NaN,e2127556f4f64592b11af22de27a7932,M,68,2018-04-26,70000.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,1.0,1.0,1.0,0.0,5.0,5.0,7.0


# Prepare data for predictive models

# Predictive Modeling

### 1. Who responds to promotional offers (including all offer types)? What characteristics are most important?

### 2. For people responding to different offers, how do their demographic characteristics differ?

### 3. How long does it take for a person to complete the offer? How does this period differ for differnt offers?

### 4. How much do people spend based on demographics and offer type?

### 5. For people who make purchases without receiving promotions, what types of offers should be sent?